# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 4 2022 12:33PM,250076,12,HH-37955,Hush Hush,Released
1,Nov 4 2022 12:33PM,250076,12,HH-37956,Hush Hush,Released
2,Nov 4 2022 12:33PM,250076,12,HH-37957,Hush Hush,Released
3,Nov 4 2022 12:33PM,250076,12,HH-37958,Hush Hush,Released
4,Nov 4 2022 12:03PM,250075,20,8732708,Encube Ethicals Inc.,Executing
5,Nov 4 2022 11:49AM,250073,10,CTF0011451-BO,"Citieffe, Inc.",Released
6,Nov 4 2022 11:48AM,250072,19,ACG-2102491770,ACG North America LLC,Released
7,Nov 4 2022 11:48AM,250071,10,CTF0011439-BO,"Citieffe, Inc.",Released
8,Nov 4 2022 11:43AM,250070,10,CTF0011287-BO,"Citieffe, Inc.",Released
9,Nov 4 2022 11:43AM,250069,10,0086144657,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,250071,Released,1
15,250072,Released,1
16,250073,Released,1
17,250075,Executing,1
18,250076,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250071,NaN,1.0
250072,NaN,1.0
250073,NaN,1.0
250075,1.0,NaN
250076,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250022,0.0,1.0
250034,1.0,0.0
250037,0.0,1.0
250040,0.0,1.0
250041,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250022,0,1
250034,1,0
250037,0,1
250040,0,1
250041,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250022,0,1
1,250034,1,0
2,250037,0,1
3,250040,0,1
4,250041,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250022,,1
1,250034,1,
2,250037,,1
3,250040,,1
4,250041,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 4 2022 12:33PM,250076,12,Hush Hush
4,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.
5,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc."
6,Nov 4 2022 11:48AM,250072,19,ACG North America LLC
7,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc."
8,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc."
9,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation
12,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation
60,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC"
61,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 4 2022 12:33PM,250076,12,Hush Hush,,4
1,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.,1,
2,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc.",,1
3,Nov 4 2022 11:48AM,250072,19,ACG North America LLC,,1
4,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc.",,1
5,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",,1
6,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,3,
7,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,,48
8,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",,1
9,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 12:33PM,250076,12,Hush Hush,4,
1,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.,,1
2,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc.",1,
3,Nov 4 2022 11:48AM,250072,19,ACG North America LLC,1,
4,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc.",1,
5,Nov 4 2022 11:43AM,250070,10,"Citieffe, Inc.",1,
6,Nov 4 2022 11:43AM,250069,10,ISDIN Corporation,,3
7,Nov 4 2022 11:40AM,250065,10,ISDIN Corporation,48,
8,Nov 4 2022 11:37AM,250067,20,"Emersa Waterbox, LLC",1,
9,Nov 4 2022 11:11AM,250063,10,"Senseonics, Incorporated",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 12:33PM,250076,12,Hush Hush,4,
1,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.,,1
2,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc.",1,
3,Nov 4 2022 11:48AM,250072,19,ACG North America LLC,1,
4,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 12:33PM,250076,12,Hush Hush,4.0,NaN
1,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.,NaN,1.0
2,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc.",1.0,NaN
3,Nov 4 2022 11:48AM,250072,19,ACG North America LLC,1.0,NaN
4,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 12:33PM,250076,12,Hush Hush,4.0,0.0
1,Nov 4 2022 12:03PM,250075,20,Encube Ethicals Inc.,0.0,1.0
2,Nov 4 2022 11:49AM,250073,10,"Citieffe, Inc.",1.0,0.0
3,Nov 4 2022 11:48AM,250072,19,ACG North America LLC,1.0,0.0
4,Nov 4 2022 11:48AM,250071,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2250566,55.0,4.0
12,250076,4.0,0.0
16,500071,1.0,1.0
18,250022,1.0,0.0
19,750153,3.0,0.0
20,500142,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2250566,55.0,4.0
1,12,250076,4.0,0.0
2,16,500071,1.0,1.0
3,18,250022,1.0,0.0
4,19,750153,3.0,0.0
5,20,500142,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,55.0,4.0
1,12,4.0,0.0
2,16,1.0,1.0
3,18,1.0,0.0
4,19,3.0,0.0
5,20,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,12,Released,4.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,4.0,0.0,1.0,0.0,0.0,1.0
Released,55.0,4.0,1.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,4.0,0.0,1.0,0.0,0.0,1.0
1,Released,55.0,4.0,1.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,4.0,0.0,1.0,0.0,0.0,1.0
1,Released,55.0,4.0,1.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()